In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold,GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
file_path = 'IRENA_RenewableEnergy_Statistics_2000-2022.csv'

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

df_irena = pd.read_csv(file_path, encoding=result['encoding'])

file_path_1 = 'organised_Gen.csv'

with open(file_path_1, 'rb') as f:
    result = chardet.detect(f.read())

df_us_data = pd.read_csv(file_path_1, encoding=result['encoding'])

file_path_2 = '02 modern-renewable-energy-consumption.csv'

with open(file_path_2, 'rb') as f:
    result = chardet.detect(f.read())

df_world_data = pd.read_csv(file_path_2, encoding=result['encoding'])

In [8]:
def run_rf(X_train, y_train):    
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
    }

    # Initialize the RandomForestRegressor
    forest_regressor = RandomForestRegressor(random_state=42)

    # Perform RandomizedSearch CV
    rand_search = RandomizedSearchCV(
        estimator=forest_regressor,
        param_distributions=param_grid,
        scoring='neg_mean_squared_error',
        cv=5,
        n_jobs=-1
    )

    rand_search.fit(X_train, y_train)

    # Extract the results
    results = pd.DataFrame(rand_search.cv_results_)
    results["MSE"] = -results["mean_test_score"]
    results["RMSE"] = np.sqrt(results["MSE"])
    results_sorted = results.sort_values("RMSE").head(10)  # Top 10 results
    
    return results_sorted

In [11]:
df_total_only = df_us_data[df_us_data["ENERGY SOURCE"] == "Total"].reset_index(drop=True)

df_total_only = df_total_only.iloc[:, 1:]

X = df_total_only.drop(columns=["GENERATION (Megawatthours)"])
y = df_total_only["GENERATION (Megawatthours)"]

categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

X_encoded = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

rf_results = run_rf(X_train, y_train)

In [12]:
rf_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,MSE,RMSE
6,32.600152,0.289370,0.267210,0.019061,100,5,None,"{'n_estimators': 100, 'min_samples_split': 5, ...",-8.307218e+11,-6.257201e+11,-8.236149e+11,-1.130252e+12,-7.045862e+11,-8.229789e+11,1.717002e+11,1,8.229789e+11,9.071819e+05
8,39.645146,2.765444,0.156838,0.013727,50,2,None,"{'n_estimators': 50, 'min_samples_split': 2, '...",-8.909273e+11,-6.310778e+11,-8.468245e+11,-1.110098e+12,-7.777640e+11,-8.513384e+11,1.565071e+11,2,8.513384e+11,9.226800e+05
2,44.606803,0.564003,0.451069,0.053303,200,10,None,"{'n_estimators': 200, 'min_samples_split': 10,...",-8.747676e+11,-7.235385e+11,-9.343759e+11,-1.395637e+12,-7.324255e+11,-9.321489e+11,2.455715e+11,3,9.321489e+11,9.654786e+05
3,22.145603,0.528482,0.215264,0.016914,100,10,None,"{'n_estimators': 100, 'min_samples_split': 10,...",-8.719455e+11,-7.366755e+11,-9.537071e+11,-1.417650e+12,-7.417792e+11,-9.443515e+11,2.504024e+11,4,9.443515e+11,9.717775e+05
1,11.715160,0.092842,0.135105,0.022317,50,10,None,"{'n_estimators': 50, 'min_samples_split': 10, ...",-8.693226e+11,-7.335703e+11,-9.372676e+11,-1.452198e+12,-7.655152e+11,-9.515747e+11,2.606676e+11,5,9.515747e+11,9.754869e+05
4,9.384230,0.082321,0.079541,0.006405,100,10,20,"{'n_estimators': 100, 'min_samples_split': 10,...",-1.715601e+12,-1.582801e+12,-1.765809e+12,-2.238200e+12,-1.507281e+12,-1.761938e+12,2.553150e+11,6,1.761938e+12,1.327380e+06
7,2.228367,0.038044,0.019385,0.001597,50,10,10,"{'n_estimators': 50, 'min_samples_split': 10, ...",-3.444162e+12,-3.431767e+12,-3.533125e+12,-4.136688e+12,-3.298594e+12,-3.568867e+12,2.936301e+11,7,3.568867e+12,1.889145e+06
0,2.115846,0.056368,0.030959,0.004467,100,2,5,"{'n_estimators': 100, 'min_samples_split': 2, ...",-9.192126e+12,-8.683420e+12,-8.838647e+12,-9.416961e+12,-9.217032e+12,-9.069637e+12,2.682846e+11,8,9.069637e+12,3.011584e+06
5,4.112280,0.045603,0.058667,0.011875,200,10,5,"{'n_estimators': 200, 'min_samples_split': 10,...",-9.228684e+12,-8.675986e+12,-8.805492e+12,-9.466687e+12,-9.198488e+12,-9.075067e+12,2.912498e+11,9,9.075067e+12,3.012485e+06
9,1.029596,0.027324,0.014855,0.002790,50,5,5,"{'n_estimators': 50, 'min_samples_split': 5, '...",-9.197237e+12,-8.719644e+12,-8.752025e+12,-9.508292e+12,-9.232721e+12,-9.081984e+12,3.026242e+11,10,9.081984e+12,3.013633e+06
